In [1]:
import matplotlib.pyplot as plt;
import numpy as np;
import numpy.random as rnd;
import scipy.linalg as sla;
import control.matlab as con;

con.use_numpy_matrix(flag=False, warn=True)

In [2]:
# Pomoćna rutina: rješavanje pomaknutog gornje-trokutastog sustava.
def tri_solve( T, s, b ):
    # Rješava (T + s*I)*x = b.
    n = np.size( T, 1 );
    x = np.zeros( n, dtype=complex );
    
    # Supstitucija unatrag.
    for i in range( n-1, -1, -1 ):
        rhs = b[i];
        for j in range( n-1, i, -1 ):
            rhs = rhs - T[i, j] * x[j];
        x[i] = rhs / ( T[i, i] + s );
        
    return x;

In [3]:
# Provjera gornje rutine.
T = np.array( [[1+2j, 2-1j, 3+2j], [0, 4-2j, 5+1j], [0, 0, 6]] );
s = 5-2j;
xx = np.array( [-2+1j, -7+2j, 5-3j] );

b = (T + s*np.eye(3)) @ xx;

x = tri_solve( T, s, b );
print( x );

[-2.+1.j -7.+2.j  5.-3.j]


In [4]:
# Implementacija Bartels-Stweartovog algoritma za rješavanje Syvesterove jednadžbe.
def BartelsStewart( A, B, C ):
    # Rješava A*X + X*B + C = 0.
    
    # A je n x n matrica, B je k x k matrica -> X je n x k matrica.
    n = np.size( A, 1 );
    k = np.size( B, 1 );
    
    X = np.zeros( ( n, k ), dtype=complex );
    
    # 1. Konverzija u Schurovu formu matrica A i B.
    # (Radi jednostavnosti, koristimo kompleksnu formu, trebalo bi realnu.)
    [T_A, Q_A] = sla.schur( A, output='complex' ); # A = Q_A * T_A * Q_A.H
    [T_B, Q_B] = sla.schur( B, output='complex' ); # B = Q_B * T_B * Q_B.H
    
    # Primijeni ortogonalne transformacije na desnu stranu.
    CC = Q_A.conj().T @ C @ Q_B;
    
    # 2. Sada rješavamo T_A * X + X * T_B + CC = 0, stupac po stupac od X.
    for l in range( 0, k ):
        # Rješavamo trokutasti sustav:
        # (A + b_ll I)x_l = -(c_l + b_1l*x1 + b_2l*x2 + ... + b_(l-1)l*x_(l-1)).
        
        # Pripremi desnu stranu trokutastog sustava.
        rhs = -CC[:, l];
        
        for i in range( 0, l ):
            rhs = rhs - T_B[i, l] * X[:, i];
            
        # Riješi trokutasti sustav (A + b_ll I) x_l = rhs.
        xx = tri_solve( T_A, T_B[l, l], rhs );
        X[:, l] = xx.reshape( -1 ); # Reshapeaj xx u vektor (-1 = pogodi jedinu preostalu dimenziju)
        
    # Primijeni nazad ortogonalne transformacije na X.
    X = Q_A @ X @ Q_B.conj().T;
    
    return X;

In [5]:
# Generiraj slučajnu Sylv. jednadžbu.
n = 10; k = 5;

A = rnd.randn( n, n );
B = rnd.randn( k, k );

XX = rnd.randn( n, k );
C = -(A @ XX + XX @ B);

# Pozovi našu rutinu za rješavanje Sylv. jednadžbe.
X = BartelsStewart( A, B, C );

# Usporedi rješenja.
print( sla.norm( XX - X, 'fro' ) );

# Riješi pomoću ugrađene funkcije lyap.
XXX = con.lyap( A, B, C );

# Usporedi rješenja.
print( sla.norm( XX - XXX, 'fro' ) );

4.7569481798118256e-14
2.3950254287050302e-14


In [6]:
# Provjerimo je li neki slučajni sustav stabilan tako da riješimo Lyap. jednadžbu
# A*P + P*A' + B*B' = 0
# Ako je rješenje P pozitivno definitno, sustav je stabilan.
# (Uoči: rješenje je simetrično ako je jedinstveno, 
# tj. ako A nema svoj. vr. centralno simetričnih s obzirom na ishodište.)
sys_t = con.rss( 10, 2, 3 ); # rss generira stabilni sustav.

A = sys_t.A;
B = sys_t.B;

P = BartelsStewart( A, A.T, B @ B.T );

# Provjerimo da li je rješenje realna, simetrična matrica.
print( 'Originalni sustav: ' ); 

print( 'norm(P-real(P)) = ', sla.norm( P - np.real( P ), 'fro' ) );
P = np.real( P );
print( 'norm(P-P.T) = ', sla.norm( P - P.T, 'fro' ) );

# Da li je pozitivno definitna? Da li je sustav stabilan?
print( 'Svojstvene vrijednosti od P:\n', sla.eigvals( P ) );
print( 'Polovi sustava:\n', con.pole( sys_t ) );

print( '\n' );

if( min( np.real( sla.eigvals(P) ) ) > 0 ):
    print( 'Rješenje Lyap. jednadžbe je poz. definitno.' );
else:
    print( 'Rješenje Lyap. jednadžbe nije poz. definitno.' );

if( max( np.real( con.pole(sys_t) ) ) < 0 ):
    print( 'Sustav je interno stabilan.' );
else:
    print( 'Sustav nije interno stabilan.' );

    
# --------------------------------------------------------------
# Odredi pomak tako da sustav više nije stabilan.
p = max( np.real( con.pole( sys_t ) ) );  # Najdesniji pol sustava je p.
A = A - (p-1)*np.eye( np.size( A, 1 ) ); # Sada je najdesniji pol sustava p-(p-1)=1.
sys_t.A = A;

# Sada sustav nije više stabilan. Uvjerimo se da niti P nije poz. def.
P = BartelsStewart( A, A.T, B @ B.T );

# Provjerimo da li je rješenje realna, simetrična matrica.
print( '\n-----\n\nPomaknuti sustav: ' ); 

print( 'norm(P-real(P)) = ', sla.norm( P - np.real( P ), 'fro' ) );
P = np.real( P );
print( 'norm(P-P.T) = ', sla.norm( P - P.T, 'fro' ) );

# Da li je pozitivno definitna? Da li je sustav stabilan?
print( 'Svojstvene vrijednosti od P:\n', sla.eigvals( P ) );
print( 'Polovi sustava:\n', con.pole( sys_t ) );

print( '\n' );

if( min( np.real( sla.eigvals(P) ) ) > 0 ):
    print( 'Rješenje Lyap. jednadžbe je poz. definitno.' );
else:
    print( 'Rješenje Lyap. jednadžbe nije poz. definitno.' );

if( max( np.real( con.pole(sys_t) ) ) < 0 ):
    print( 'Sustav je interno stabilan.' );
else:
    print( 'Sustav nije interno stabilan.' );

# Koliko ima polova u lijevoj poluravnini, a koliko svojstvenih vrijednosti od P u desnoj?
# Uočite da TM sa predavanja nešto govori u slučaju M>0, a kod nas je samo M=BB*>=0.

Originalni sustav: 
norm(P-real(P)) =  2.9145666187216794e-13
norm(P-P.T) =  3.356066948717061e-12
Svojstvene vrijednosti od P:
 [1.02327902e+02+0.j 3.17663571e+01+0.j 2.46214156e+00+0.j
 1.05761357e+00+0.j 7.95488435e-01+0.j 2.92286387e-01+0.j
 8.72410278e-02+0.j 3.25231335e-02+0.j 8.72833575e-04+0.j
 5.82576750e-04+0.j]
Polovi sustava:
 [-1.12065007+20.33525388j -1.12065007-20.33525388j
 -0.40745574 +0.j         -0.78478246 +0.j
 -2.00246603 +0.72240969j -2.00246603 -0.72240969j
 -2.61309919 +0.70769686j -2.61309919 -0.70769686j
 -2.21078764 +0.j         -2.39772151 +0.j        ]


Rješenje Lyap. jednadžbe je poz. definitno.
Sustav je interno stabilan.

-----

Pomaknuti sustav: 
norm(P-real(P)) =  9.10835369461708e-11
norm(P-P.T) =  1.1174773757788113e-08
Svojstvene vrijednosti od P:
 [ 7.03928003e+03+0.j -3.86875382e+03+0.j -4.10527063e+01+0.j
  1.58866289e+01+0.j  5.40144449e+00+0.j -3.13086380e+00+0.j
 -2.10821204e+00+0.j  1.23551547e+00+0.j  2.30252798e-01+0.j
  2.09004871e-02+0.